## Imports ##

In [ ]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
#Loading Dataset
DATASET_PATH = "../Datasets/Covid_Cleaned.csv"
COVID_data = pd.read_csv(DATASET_PATH)

## Batch preperation ##

In [ ]:
X = COVID_data.drop(columns=['CLASIFFICATION_FINAL'])  # features (21)
y = COVID_data['CLASIFFICATION_FINAL']                 # target (low / high risk of COVID-19)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42) # 70% train, 30% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) # test -> 15% validation, 15% final test

In [ ]:
# Creating Pytorch Tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [ ]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [ ]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=37) # 70% train, 30% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=37) # test -> 15% validation, 15% final test

## Model Training ##

In [ ]:
import torch.nn as nn
import torch.optim as optim
from src.utils import OurModel


In [ ]:
ml = OurModel()

In [ ]:
# model configuration
config = nn.Sequential(
    nn.Linear(21, 21),
    nn.ReLU(),
    nn.Linear(21, 21),
    nn.Tanh(),
    nn.Linear(21, 21),
    nn.ReLU(),
    nn.Linear(21, 2),
)
ml.add_configuration(config)

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.RMSprop(ml.parameters(), lr=0.001)

In [ ]:
#Model Training

epochs = range(20)
for ep in epochs:
    OurModel.train()
    sum_loss = 0
    for input, true in train_loader:
        y_pred = ml(input)
        loss = criterion(y_pred, true)
        optimizer.zero_grad() # Reseting gradient
        loss.backward()
        optimizer.step() # updating
        sum_loss += loss.item()
